In [1]:
import pandas as pd
import numpy as np

In [2]:
# Data: 
xls_dict  = pd.read_excel('data/trading-game-data-15112023.xlsx', sheet_name=None)

info_df = xls_dict['info'][['Ticker', 'RBICS Economy']]
info_df = info_df.set_index('Ticker')

index_price_df = xls_dict['index-price']
price_df = xls_dict['price']
size_df = xls_dict['size']
price_to_book_df = xls_dict['price-to-book']
turnover_df = xls_dict['turnover']

price_df = xls_dict['price'].reset_index()
price_df = price_df.drop('index', axis = 1)
price_df['Date'] = pd.to_datetime(price_df['Date'])
price_df.set_index('Date', inplace=True)

In [70]:
def calculate_monthly_returns(price_df):
    # Assuming price_df is indexed by date and contains end-of-day prices
    monthly_returns = np.log(price_df / price_df.shift(1))
    return monthly_returns

log_returns = calculate_monthly_returns(price_df)

In [65]:
sentiment_index = pd.read_excel("data/michigan index.xlsx", sheet_name='Sheet1')
sentiment_index['Date'] = sentiment_index['Year'].astype(str).str.strip() + '-' + sentiment_index['Month'].astype(str).str.strip() + '-01'
sentiment_index['Date'] = pd.to_datetime(sentiment_index['Date'], format='%Y-%m-%d')
sentiment_index.set_index('Date', inplace=True)


# Drop the individual "Month" and "Year" columns if no longer needed
sentiment_index.drop(['Month', 'Year'], axis=1, inplace=True)
ff_data = pd.read_csv("data/F-F_Momentum_Factor_daily.csv")
ff_data = ff_data.rename(columns={"Unnamed: 0": "Date"})
ff_data['Date'] = pd.to_datetime(ff_data['Date'], format='%Y%m%d')
ff_data.set_index('Date', inplace=True)
ff_data.columns

Index(['Mom   '], dtype='object')

In [73]:
combined_data

,Index,Mom,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1926-11-03,NaN,0.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-04,NaN,-0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-05,NaN,1.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-06,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-08,NaN,-0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-08,NaN,NaN,-0.010458,0.021886,0.005868,-0.000915,-0.028652,-0.000633,-0.015219,0.008310,...,-0.003009,-0.012615,-0.012359,-0.025679,0.000413,0.001035,-0.010152,-0.007721,-0.010021,0.021952
2023-11-09,NaN,NaN,-0.015199,-0.021886,-0.002628,-0.028495,-0.018355,-0.012640,0.006517,-0.011392,...,-0.024518,-0.006453,0.000291,-0.015657,0.000103,-0.007191,-0.007082,-0.019171,-0.036499,-0.001406
2023-11-10,NaN,NaN,0.006753,0.004246,0.021638,0.003976,0.022511,0.004480,0.015004,0.017258,...,-0.058607,0.008482,0.007644,0.040983,0.016797,0.009894,0.012408,0.017911,0.011303,-0.007707


In [72]:
import statsmodels.api as sm

# Combine sentiment, momentum, and log returns data
combined_data = pd.concat([sentiment_index, ff_data['Mom   '], log_returns], axis=1)

# Perform regression with missing data handling
model = sm.OLS(combined_data.loc[], combined_data[['Sentiment', 'Mom   ']], missing='drop').fit()


KeyError: 'Log_Return'

In [34]:



def calc_sentiment_outsample(price_df):
    log_returns = np.log(price_df / price_df.shift(1))

    n_obs = len(log_returns)
    # Calculate volatility (of the )
    volatility = log_returns.rolling(window=6*21).std() * np.sqrt(n_obs)
    volatility_mean = volatility.mean()

    # Step 2: Sort stocks and create buckets
    sorted_stocks = volatility_mean.sort_values()
    buckets = np.array_split(sorted_stocks, 10)  # Splitting into 10 buckets

    # Step 3: using the returns; 
    monthly_log_returns = calculate_monthly_returns(price_df)

    return buckets, monthly_log_returns
    

def calculate_future_returns(current_prices, future_prices):
    # Current prices are 
    return np.log(future_prices/current_prices)



buckets = calc_sentiment_outsample(price_df)


                   A       AAL      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2022-12-31       NaN       NaN       NaN       NaN       NaN       NaN   
2023-01-31  0.016107  0.238125  0.104829 -0.089664  0.262004  0.006898   
2023-02-28 -0.068791 -0.009963  0.021393  0.040781  0.103937 -0.083244   
2023-03-31 -0.025901 -0.080095  0.112121  0.034926  0.009044 -0.004532   
2023-04-30 -0.021260 -0.078236  0.028575 -0.053154 -0.038764  0.087053   
2023-05-31 -0.157714  0.080268  0.043647 -0.091110 -0.086351 -0.079771   
2023-06-30  0.038832  0.193758  0.090142 -0.023691  0.154892  0.066559   
2023-07-31  0.012561 -0.068635  0.012704  0.104559  0.171850  0.020967   
2023-08-31 -0.005765 -0.128512 -0.044658 -0.017671 -0.145743 -0.078741   
2023-09-30 -0.079478 -0.139660 -0.092859  0.014188  0.042126 -0.060594   
2023-10-31 -0.078576 -0.138787 -0.002573 -0.054313 -0.148373 -0.024035   
2023-11-30  0.067063  0.052414  0.0627